In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.datasets import imdb
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

import import_notebook
from readWrite import ReadWrite
from vectorToDoc import VectorToDoc

Using TensorFlow backend.


importing Jupyter notebook from readWrite.ipynb
importing Jupyter notebook from vectorToDoc.ipynb
importing Jupyter notebook from dictionary.ipynb
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\NielPC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\NielPC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
tweetFile = "combineVectors.csv"
tweetClassFile = "combineVectorsResult.txt"

readWrite = ReadWrite()
xdata = readWrite.readFile(tweetFile)
ydata = readWrite.readFileClassifier(tweetClassFile)

x_train = xdata[:int(len(xdata) * 0.8) + 1]
y_train = ydata[:int(len(ydata) * 0.8) + 1]
x_test = xdata[int(len(xdata) * 0.8):-1]
y_test = ydata[int(len(ydata) * 0.8):-1]

print("x_train: {}, shape: {}".format(len(x_train), x_train.shape))
print("y_train: {}, shape: {}".format(len(y_train), y_train.shape))
print("x_test: {}, shape: {}".format(len(x_test), x_test.shape))
print("y_test: {}, shape: {}".format(len(y_test), y_test.shape))

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(ydata)
encoded = encoder.transform(ydata)
# convert integers to dummy variables (i.e. one hot encoded)
encoding = np_utils.to_categorical(encoded)

encoding_train = encoding[:int(len(xdata) * 0.8) + 1]
encoding_test = encoding[int(len(ydata) * 0.8):-1]
print("encoding_train shape: {}".format(encoding_train.shape))
print("encoding_test shape: {}".format(encoding_test.shape))

Read complete, total records: 1125
Read complete, total records: 1125
x_train: 901, shape: (901, 20)
y_train: 901, shape: (901,)
x_test: 224, shape: (224, 20)
y_test: 224, shape: (224,)
encoding_train shape: (901, 3)
encoding_test shape: (224, 3)


In [5]:
vocabSize = 2000
outputDim = 64
recordsSize = 20

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

model = Sequential()
model.add(Embedding(vocabSize, outputDim, input_length=recordsSize))
# model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, input_dim=(20,), activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# print(model.summary())

In [6]:
batch_size = outputDim
epochs = 30

model.fit(x_train, encoding_train, validation_data=(x_test, encoding_test), epochs=epochs, batch_size=outputDim)
scores = model.evaluate(x_test, encoding_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 901 samples, validate on 224 samples
Epoch 1/30
901/901 [==============================] - 2s - loss: 0.5305 - acc: 0.7333 - val_loss: 0.2057 - val_acc: 0.9375
Epoch 2/30
901/901 [==============================] - 0s - loss: 0.4083 - acc: 0.8524 - val_loss: 0.2588 - val_acc: 0.9375
Epoch 3/30
901/901 [==============================] - 0s - loss: 0.3683 - acc: 0.8561 - val_loss: 0.2317 - val_acc: 0.9405
Epoch 4/30
901/901 [==============================] - 0s - loss: 0.3085 - acc: 0.8724 - val_loss: 0.1889 - val_acc: 0.9375
Epoch 5/30
901/901 [==============================] - 0s - loss: 0.2337 - acc: 0.9038 - val_loss: 0.1708 - val_acc: 0.9449
Epoch 6/30
901/901 [==============================] - 0s - loss: 0.1933 - acc: 0.9057 - val_loss: 0.1942 - val_acc: 0.9315
Epoch 7/30
901/901 [==============================] - 0s - loss: 0.1636 - acc: 0.9046 - val_loss: 0.2208 - val_acc: 0.9167
Epoch 8/30
901/901 [==============================] - 0s - loss: 0.1436 - acc: 0.9190 - val_l

In [7]:
vectorsArr2 = readWrite.readFile('docVectors3.csv')
oriRecords2 = readWrite.readOriFile('user_tweets3.txt')

classResult = []
predicted = model.predict_classes(vectorsArr2)
predicted = np.reshape(predicted, (predicted.size,))

print()
print(predicted)

for idx, score in enumerate(predicted):    
    if(score >= 1): 
        classResult.append(oriRecords2[idx])       
        print(predicted[idx])
        print(oriRecords2[idx])        

Read complete, total records: 370
320/370 [========================>.....] - ETA: 0s
[0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 2 0 0 0 2 0 0 0 0 0 0 0 0 0 2 2 0 0 0 0 2 0 2 0 2 0 0 0 0 0 0 0
 0 0 0 0 2 0 0 2 0 0 0 2 0 0 0 2 0 0 0 0 2 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 1 1 0 0 0 2 0 0 0 0 0 0 0 2 0 2 2 0 0 0 0 0 0 0 0 0 0 0 2 2 0 0 0 0 0 0 0
 0 0 0 0 2 2 0 2 0 0 2 0 0 0 0 0 0 0 0 0 0 2 1 1 1 0 0 0 0 0 0 2 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 2 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 2 0 1 0 0 0 0 0
 0 0 0 0 0 2 0 0 0 0 0 0 1 0 0 0 0 0 0 2 0 1 2 0 0 0 0 2 1 0 0 0 0 2 0 2 0
 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 2 0 0 0 1 0 2
 0 0 0 1 0 2 0 2 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 2]
2
@SMRT_Singapore is there a track fault?? Why is your train stuck at Tanah Merah towards Pasir Ris

2
@SMRT_Singapore EWL, West bound, train is stopping more than 